# Modeling a Zombie Apocalypse in PySB

## Credits
* Zombie Apocalypse example by [Munz et al. 2009](https://mysite.science.uottawa.ca/rsmith43/Zombies.pdf)
* SciPy implementation by [Christopher Campo](https://scipy-cookbook.readthedocs.io/items/Zombie_Apocalypse_ODEINT.html)
* Jupyter Notebook interactivity and PySB conversion by Alex Lubbock

## Background

This example demonstrates how to solve a system of first order ODEs using [PySB](https://pysb.org). In this lighthearted example, a system of ODEs can be used to model a "zombie invasion", using the equations specified in Munz et al. 2009.

![asdf](model-diagram.png)

Populations (the "species" of the model, represented as `Monomers` in PySB):

    Susceptible: the number of susceptible victims
    Zombie: the number of zombies
    Removed: the number of people "killed"
    
Parameters:

    k_birth (π in the figure above): the population birth rate
    k_death (δ in the figure above): the chance of a natural death
    k_infect (β in the figure above): the chance the "zombie disease" is transmitted (an alive person becomes a zombie)
    k_resurrect (γ in the figure above): the chance a dead person is resurrected into a zombie
    k_destroy (α in the figure above): the chance a zombie is totally destroyed

This involves solving a system of first order ODEs given by: dy/dt = f(y, t). However, using PySB, the system can be given as a set of reaction rules and the underlying ODE system is solved at runtime. This representation is typically clearer and easier to update and maintain.

## How to run this notebook

Select the **Kernel** menu at the top of the page, then select **Restart & Run All**. After a few seconds, a plot should appear at the bottom of the page. Adjust the slider bars to run a new ODE simulation and update the plot.

In [ ]:
# Enable interactive plots in Jupyter Notebook
%matplotlib notebook

# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from pysb import Model, Monomer, Parameter, Initial, Rule
from pysb.simulator import ScipyOdeSimulator

# Create an empty model
Model()

# Default parameter values
Parameter('k_birth', 0)       # birth rate
Parameter('k_death', 0.0001)  # natural death rate (per day)
Parameter('k_infect', 0.0095)  # transmission rate  (per day)
Parameter('k_resurrect', 0.0001)  # resurrect rate (per day)
Parameter('k_destroy', 0.0001)  # destroy rate  (per day)

# Our "monomers" are our types of individual
# Analagous to chemical species in a reaction model
Monomer('Susceptible')
Monomer('Zombie')
Monomer('Removed')

# Initial conditions
Initial(Susceptible(), Parameter('S_0', 500))    # initial population
Initial(Zombie(), Parameter('Z_0', 0))           # initial zombie population
Initial(Removed(), Parameter('R_0', 0))          # initial death population

# Reaction rules
Rule('r_birth', None >> Susceptible(), k_birth)
Rule('r_death', Susceptible() >> Removed(), k_death)
Rule('r_infected', Susceptible() + Zombie() >> Zombie() + Zombie(), k_infect)
Rule('r_destroy', Susceptible() + Zombie() >> Susceptible() + Removed(), k_destroy)
Rule('r_resurrect', Removed() >> Zombie(), k_resurrect)

# Time vector (0 to 5 days, 1000 steps)
tspan = np.linspace(0, 5, 1000)

# Create the simulator
sim = ScipyOdeSimulator(model, tspan)

# Run the simulation
soln = sim.run()

# Plot results
fig, ax = plt.subplots()
l1, = plt.plot(tspan, soln.species[:, 0], label='Living')
l2, = plt.plot(tspan, soln.species[:, 1], label='Zombies')
plt.xlabel('Days from outbreak')
plt.ylabel('Population')
plt.title('Zombie Apocalypse')
plt.legend(loc=0)

# Adjust the main plot to make room for the sliders
plt.subplots_adjust(bottom=0.25)

# Add interactivity
axfreq = plt.axes([0.25, 0.1, 0.65, 0.03])
s0_slider = Slider(
    ax=axfreq,
    label='S0: Init. human pop.',
    valmin=0,
    valmax=1000,
    valinit=S_0.value,
)
ax_z0 = plt.axes([0.25, 0.05, 0.65, 0.03])
z0_slider = Slider(
    ax=ax_z0,
    label='Z0: Init. zombie pop.',
    valmin=0,
    valmax=1000,
    valinit=Z_0.value,
)

# Callback function to re-plot new initial conditions
def run_ode(_):
    soln = sim.run(initials={
        Susceptible(): s0_slider.val,
        Zombie(): z0_slider.val
    })
    
    l1.set_ydata(soln.species[:, 0])
    l2.set_ydata(soln.species[:, 1])
    fig.canvas.draw_idle()

s0_slider.on_changed(run_ode)
z0_slider.on_changed(run_ode)